# Project 3: Data Cleaning - Tidy up messy Datasets (Movies Dataset)

# Project Brief for Self-Coders

Here you´ll have the opportunity to code major parts of Project 3 on your own. If you need any help or inspiration, have a look at the Videos or the Jupyter Notebook with the full code. <br> <br>
Keep in mind that it´s all about __getting the right results/conclusions__. It´s not about finding the identical code. Things can be coded in many different ways. Even if you come to the same conclusions, it´s very unlikely that we have the very same code. 

## First Steps 

1. __Load__ and __inspect__ the messy dataset __movies_metadata.csv__. Identify columns with nested / stringified json data.

In [1]:
import pandas as pd
import numpy as np

In [351]:
mm_df = pd.read_csv('movies_metadata.csv',low_memory=False)

In [ ]:
mm_df.head(5)

In [ ]:
mm_df.info()

In [ ]:
mm_df.describe()

In [ ]:
mm_df.columns.to_list()

In [ ]:
mm_df.shape

In [ ]:
def parse_for_bool(val,filler_val=np.nan):
    true_strs=['True','true']
    false_strs=['False','false']
    if val in true_strs:
        return True
    elif val in false_strs:
        return False
    else:
        return filler_val
    

## Dropping irrelevant Columns

2. __Drop__ the irrelevant columns 'adult', 'imdb_id', 'original_title', 'video' and 'homepage'.

In [257]:
mm_df.drop(columns=['adult','imdb_id','original_title', 'video' , 'homepage'],inplace=True)

In [ ]:
mm_df.info()

In [ ]:
mm_df.describe()

## How to handle stringified JSON columns

3. __Evaluate__ Python Expressions in the stringified columns ["belongs_to_collection", "genres", "production_countries", "production_companies", "spoken_languages"] and __remove quotes__ ("") where possible.

In [169]:
import json
import re

In [170]:
def clean_json_null(json_str):
    return re.sub(':\s?(None|Nan|nan)\s?([,}])',lambda m:': null'+m.group(2),json_str)

def remove_json_str_sing_quote(json_str):
    
    
    json_key_pat = "\'([a-zA-Z_0-9]+)\'\s?:"
    json_data_pat = ":\s?\'([^']*)\'?([,}])"
    json_data_double_quo_remove_pat = '(:\s?\"[^(,})]*)(\")([^(,})]*\"[,}])'
    json_invalid_esc_pat = "\\x"
    
    json_str = re.sub(json_key_pat,lambda m:'"'+m.group(1)+'"'+':',json_str)
    json_str = re.sub(json_data_pat,lambda m: ':'+'"'+m.group(1)+'"'+m.group(2),json_str)
    
    while re.match('.*'+json_data_double_quo_remove_pat+'.*',json_str):
        json_str = re.sub(json_data_double_quo_remove_pat,lambda m: m.group(1)+"'"+m.group(3),json_str)
        
    json_str = json_str.replace(json_invalid_esc_pat,'')
    
        
    return json_str

In [171]:
def process_for_json(data):
    try:
        if pd.isna(data) or ((re.match('^\[.*\]$',data) == None) and (re.match('^\{.*\}$',data) == None)):
            return np.nan

        data = remove_json_str_sing_quote(data)
        data = clean_json_null(data)
        return json.loads(data)
    except:
        return np.nan

In [258]:
mm_df['belongs_to_collection'] = mm_df['belongs_to_collection'].apply(process_for_json)
mm_df['genres'] = mm_df['genres'].apply(process_for_json)
mm_df['spoken_languages'] = mm_df['spoken_languages'].apply(process_for_json)
mm_df['production_countries'] = mm_df['production_countries'].apply(process_for_json)
mm_df['production_companies'] = mm_df['production_companies'].apply(process_for_json)

## How to flatten nested Columns

In [173]:
def get_key_from_json(data,key,separator='|'):
#     print(pd.isna(data))
#     print(data)
    if type(data) == list:
        return separator.join(list(map(lambda x:str(x[key]),data)))   
    elif (type(data) == dict) and (key in data.keys()):
        return data[key]
    return np.nan

In [174]:
def flatten_json(data):
    return get_key_from_json(data,'name','|')

4. __Extract__ only the __collection name__ from the column "belongs_to_collection" and __overwrite__ "belongs_to_collection". <br> For example: The value in the first row (Toy Story) should be 'Toy Story Collection'.

In [259]:
mm_df['belongs_to_collection'] = mm_df['belongs_to_collection'].apply(flatten_json)
mm_df['belongs_to_collection']

0                  Toy Story Collection
1                                   NaN
2             Grumpy Old Men Collection
3                                   NaN
4        Father of the Bride Collection
                      ...              
45461                               NaN
45462                               NaN
45463                               NaN
45464                               NaN
45465                               NaN
Name: belongs_to_collection, Length: 45466, dtype: object

5. __Extract__ all __genre names__ from the column "genres" and __overwrite__ "genres". If a movie has more than one genre, __seperate genres by a pipe__ "|".<br>
For example: The value in the first row (Toy Story) should be 'Animation|Comedy|Family'.

In [260]:
mm_df['genres'] = mm_df['genres'].apply(flatten_json)
mm_df['genres']

0         Animation|Comedy|Family
1        Adventure|Fantasy|Family
2                  Romance|Comedy
3            Comedy|Drama|Romance
4                          Comedy
                   ...           
45461                Drama|Family
45462                       Drama
45463       Action|Drama|Thriller
45464                            
45465                            
Name: genres, Length: 45466, dtype: object

6. __Extract__ all __spoken language names__ from the column "spoken_languages" and __overwrite__ "spoken_languages". If a movie has more than one spoken language, __seperate spoken languages by a pipe__ "|".<br>
For example: The value in the first row (Toy Story) should be 'English'.

In [261]:
mm_df['spoken_languages'] = mm_df['spoken_languages'].apply(flatten_json)
mm_df['spoken_languages']

0                 English
1        English|Français
2                 English
3                 English
4                 English
               ...       
45461               فارسی
45462                    
45463             English
45464                    
45465             English
Name: spoken_languages, Length: 45466, dtype: object

7. __Extract__ all __production countries names__ from the column "production_countries" and __overwrite__ "production_countries". If a movie has more than one production country, __seperate production countries by a pipe__ "|".<br>
For example: The value in the first row (Toy Story) should be 'United States of America'.

In [262]:
mm_df['production_countries'] = mm_df['production_countries'].apply(flatten_json)
mm_df['production_countries']

0        United States of America
1        United States of America
2        United States of America
3        United States of America
4        United States of America
                   ...           
45461                        Iran
45462                 Philippines
45463    United States of America
45464                      Russia
45465              United Kingdom
Name: production_countries, Length: 45466, dtype: object

8. __Extract__ all __production companies names__ from the column "production_companies" and __overwrite__ "production_companies". If a movie has more than one production company, __seperate production companies by a pipe__ "|".<br>
For example: The value in the first row (Toy Story) should be 'Pixar Animation Studios'

In [263]:
mm_df['production_companies'] = mm_df['production_companies'].apply(flatten_json)
mm_df['production_companies']

0                                  Pixar Animation Studios
1        TriStar Pictures|Teitler Film|Interscope Commu...
2                              Warner Bros.|Lancaster Gate
3                   Twentieth Century Fox Film Corporation
4                Sandollar Productions|Touchstone Pictures
                               ...                        
45461                                                     
45462                                          Sine Olivia
45463                              American World Pictures
45464                                            Yermoliev
45465                                                     
Name: production_companies, Length: 45466, dtype: object

9. __Inspect__ all columns above with value_counts(). Do you see anything strange? __Take reasonable measures__!

In [ ]:
print(mm_df['production_companies'].value_counts(dropna=False).to_frame())
print(mm_df['production_countries'].value_counts(dropna=False).to_frame())
print(mm_df['spoken_languages'].value_counts(dropna=False).to_frame())
print(mm_df['genres'].value_counts(dropna=False).to_frame())
print(mm_df['belongs_to_collection'].value_counts(dropna=False).to_frame())

## Cleaning Numerical Columns

10. __Convert__ the datatype in the columns __"budget"__, __"id"__ and __"popularity"__ __to numeric__. Set invalid values as NaN.

In [180]:
def to_float(data):
    try:
        return float(data)
    except:
        return np.nan

In [264]:
mm_df['budget'] = mm_df['budget'].apply(to_float)
mm_df.id = mm_df.id.apply(to_float)
mm_df.popularity = mm_df.popularity.apply(to_float)

11. __Analyze__ the columns __"budget"__ and __"revenue"__ and __"runtime"__. Analyze movies with a budget/revenue/runtime of 0. Do you think the value 0 is the most appropriate value? __Take reasonable measures__! 

In [187]:
def zero_to_nan(data):
    if data == 0.0:
        return np.nan
    else:
        return data

In [265]:
mm_df['budget'] = mm_df['budget'].apply(zero_to_nan)
mm_df['budget'].value_counts(dropna=False)

NaN           36576
5000000.0       286
10000000.0      259
20000000.0      243
2000000.0       242
              ...  
2115000.0         1
1590000.0         1
1978000.0         1
1182273.0         1
2135161.0         1
Name: budget, Length: 1223, dtype: int64

In [233]:
mm_df['id'] = mm_df['id'].apply(zero_to_nan)
mm_df['id'].value_counts(dropna=False)

NaN         3
141971.0    3
110428.0    2
14788.0     2
84198.0     2
           ..
24740.0     1
9737.0      1
38726.0     1
71772.0     1
226693.0    1
Name: id, Length: 45434, dtype: int64

In [266]:
mm_df['popularity'] = mm_df['popularity'].apply(zero_to_nan)
mm_df['popularity'].value_counts(dropna=False)

NaN         72
0.000001    56
0.000308    43
0.000220    40
0.001177    38
            ..
1.880791     1
0.531305     1
2.424056     1
7.772822     1
1.691634     1
Name: popularity, Length: 43757, dtype: int64

12. The columns "budget" and "revenue" shall show values in Million USD. __Convert and Overwrite__!

In [267]:
mm_df['budget'] = mm_df['budget'].apply(lambda x:x/1000000)
mm_df['budget'].value_counts(dropna=False)

NaN          36576
5.000000       286
10.000000      259
20.000000      243
2.000000       242
             ...  
0.313411         1
0.270000         1
10.495000        1
23.600000        1
0.672000         1
Name: budget, Length: 1223, dtype: int64

In [236]:
mm_df['revenue'] = mm_df['revenue'].apply(lambda x:x/1000000)
mm_df['revenue'].value_counts(dropna=False)

0.000000      38052
12.000000        20
10.000000        19
11.000000        19
2.000000         18
              ...  
74.134790         1
46.236000         1
0.265318          1
1.310673          1
186.883563        1
Name: revenue, Length: 6864, dtype: int64

13. __Analyze__ movies with a __vote_count of 0__. What´s the __vote_average__ for those movies? Do you think this value is the most appropriate value? __Take reasonable measures__!

In [268]:
def fix_vote_average(row):
    if row['vote_count'] == 0:
        row['vote_average'] = np.nan
    return row

In [269]:
mm_df = mm_df.apply(fix_vote_average,axis=1)

In [270]:
mm_df[mm_df.vote_count == 0][['vote_count','vote_average']]

,vote_count,vote_average
83,0.0,NaN
107,0.0,NaN
126,0.0,NaN
132,0.0,NaN
137,0.0,NaN
...,...,...
45432,0.0,NaN
45434,0.0,NaN
45452,0.0,NaN
45464,0.0,NaN


## Cleaning DateTime Columns

14. __Convert__ the datatype in the column __"release_date"__ __to datetime__. Set invalid values as NaN.

In [271]:
# ts_nan_count = 0
def to_date_time(data):
    global ts_nan_count
    try:
        return pd.to_datetime(data)
    except:
#         ts_nan_count = ts_nan_count + 1
#         print(data)
        return np.nan

In [272]:
mm_df.release_date = mm_df.release_date.apply(to_date_time)

In [274]:
mm_df['release_date'].value_counts(dropna=False)

2008-01-01    136
2009-01-01    121
2007-01-01    118
2005-01-01    111
2006-01-01    101
             ... 
1993-05-05      1
1951-02-22      1
1913-10-27      1
1955-02-23      1
1949-10-01      1
Name: release_date, Length: 17334, dtype: int64

## Cleaning Text / String Columns

15. __Analyze__ the text columns "overview" and "tagline". Try to identify __missing data that is not represented by NaN__ (e.g. "No Data"). __Replace as NaN__ (np.nan)!

In [314]:
data =  'No overview found'
(('overview' in data) or ('Overview' in data))

True

In [321]:
def nan_no_overview(data):
    if pd.isna(data):
        return data
    elif ((('No' in data) or ('No' in data)) and (('overview' in data) or ('Overview' in data))):
        return np.nan
    elif data.strip() == '':
        return np.nan
    return data

In [324]:
mm_df['overview'] = mm_df['overview'].apply(nan_no_overview).value_counts(dropna=False).to_frame()

In [342]:
mm_df[(mm_df['tagline'].str.len() < 4)].tagline.value_counts(dropna=False).to_frame()

,tagline
-,4
Run,1
To,1
Go!,1
,1
B&W,1


## Removing Duplicates

16. __Identify__ and __remove__ duplicates!

In [345]:
mm_df[mm_df.duplicated(keep =  False)].sort_values(by = "id")

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
7345,NaN,NaN,Crime|Drama|Thriller,5511.0,fr,NaN,9.091288,/cvNW8IXigbaMNo4gKEIps0NGnhA.jpg,Fida cinematografica|Compagnie Industrielle et...,France|Italy,1967-10-25,39481.0,105.0,Français,Released,There is no solitude greater than that of the ...,Le Samouraï,7.9,187.0
9165,NaN,NaN,Crime|Drama|Thriller,5511.0,fr,NaN,9.091288,/cvNW8IXigbaMNo4gKEIps0NGnhA.jpg,Fida cinematografica|Compagnie Industrielle et...,France|Italy,1967-10-25,39481.0,105.0,Français,Released,There is no solitude greater than that of the ...,Le Samouraï,7.9,187.0
24844,NaN,NaN,Comedy|Drama,11115.0,en,NaN,6.880365,/kHaBqrrozaG7rj6GJg3sUCiM29B.jpg,Andertainment Group|Crescent City Pictures|Tag...,United States of America,2008-01-29,0.0,85.0,English,Released,NaN,Deal,5.2,22.0
14012,NaN,NaN,Comedy|Drama,11115.0,en,NaN,6.880365,/kHaBqrrozaG7rj6GJg3sUCiM29B.jpg,Andertainment Group|Crescent City Pictures|Tag...,United States of America,2008-01-29,0.0,85.0,English,Released,NaN,Deal,5.2,22.0
22151,NaN,NaN,Action|Horror|Science Fiction,18440.0,en,NaN,1.436085,/tWCyKXHuSrQdLAvNeeVJBnhf1Yv.jpg,,United States of America,2007-01-01,0.0,89.0,English,Released,NaN,Days of Darkness,5.0,5.0
14000,NaN,NaN,Action|Horror|Science Fiction,18440.0,en,NaN,1.436085,/tWCyKXHuSrQdLAvNeeVJBnhf1Yv.jpg,,United States of America,2007-01-01,0.0,89.0,English,Released,NaN,Days of Darkness,5.0,5.0
8068,NaN,NaN,Adventure|Animation|Drama|Action|Foreign,23305.0,en,NaN,1.967992,/9GlrmbZO7VGyqhaSR1utinRJz3L.jpg,Filmfour,France|Germany|India|United Kingdom,2001-09-23,0.0,86.0,हिन्दी,Released,NaN,The Warrior,6.3,15.0
9327,NaN,NaN,Adventure|Animation|Drama|Action|Foreign,23305.0,en,NaN,1.967992,/9GlrmbZO7VGyqhaSR1utinRJz3L.jpg,Filmfour,France|Germany|India|United Kingdom,2001-09-23,0.0,86.0,हिन्दी,Released,NaN,The Warrior,6.3,15.0
17229,NaN,NaN,Drama,25541.0,da,NaN,2.587911,/q19Q5BRZpMXoNCA4OYodVozfjUh.jpg,,Sweden|Denmark,2009-10-21,0.0,90.0,Dansk,Released,NaN,Brotherhood,7.1,21.0
23044,NaN,NaN,Drama,25541.0,da,NaN,2.587911,/q19Q5BRZpMXoNCA4OYodVozfjUh.jpg,,Sweden|Denmark,2009-10-21,0.0,90.0,Dansk,Released,NaN,Brotherhood,7.1,21.0


In [347]:
mm_df.shape

(45466, 19)

In [350]:
mm_df.drop_duplicates(subset = "id", inplace = False).shape

(45434, 19)

## Handling Missing Values & Removing Observations

17. __Drop__ all rows/movies with unknown __id__ or __title__.

18. __Keep__ only those rows/movies in the df with __10 or more non-NaN__ values.

## Final (Cleaning) Steps

19. __Keep__ only those rows/movies in the df with __status "Released"__. Then __drop__ the column "status".

20. The Order of the columns should be as follows: 

In [ ]:
["id", "title", "tagline", "release_date", "genres", "belongs_to_collection", 
"original_language", "budget_musd", "revenue_musd", "production_companies",
"production_countries", "vote_count", "vote_average", "popularity", "runtime",
"overview", "spoken_languages", "poster_path"]

21. __Reset__ the Index and create a __RangeIndex__.

22. __Save__ the cleaned dataset in a __csv-file__.

# +++++++++ See some Hints below +++++++++++++

# ++++++++++++++++ Hints++++++++++++++++++++

__Hints for 3.__ <br>
apply ast.literal_eval() on all stringified elements (you have to import ast):

In [ ]:
# example:
df.stringified_column = df.stringified_column.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else np.nan)

__Hints for 4., 5., 6., 7., 8.__<br> 
apply an appropriate lambda function on all column elements

__Hints for 9.__<br>
Replace all __""__ (empty strings) in the above columns by NaN (__np.nan__)

__Hints for 10.__<br>
Use pd.to_numeric() and "coerce" errors

__Hints for 11.__<br>
Replace the value 0 by NaN (__np.nan__)

__Hints for 13.__<br>
Replace the value 0 by NaN (__np.nan__)

__Hints for 14.__<br>
Use pd.to_datetime() and "coerce" errors

__Hints for 16.__<br>
There cannot be two or more movies with the same movie id.